# Cellule 0 — Setup & dossiers

In [68]:
from pathlib import Path
import re, json, hashlib
import pandas as pd

ROOT = Path(".")
DATA = ROOT / "data"
RAW  = DATA / "raw"
CLEAN = DATA / "clean"

RAW.mkdir(parents=True, exist_ok=True)
CLEAN.mkdir(parents=True, exist_ok=True)

DOCS_CSV = DATA / "docs.csv"
CHUNKS_JSONL = DATA / "chunks.jsonl"

print("OK:", DATA.resolve())

OK: H:\Documents\cyTech\Ing3\NLP\projet-nlp-rag\data\data


# Cellule 1 — Convention de métadonnées (schéma)

In [69]:
DOC_SCHEMA = [
    "doc_id", "title", "date", "author",
    "source", "url", "local_path", "language"
]

CHUNK_SCHEMA = [
    "chunk_id", "doc_id", "text",
    "title", "date", "author", "source", "url",
    "start_char", "end_char"
]

# Cellule 2 — Ajouter des docs (manuel au début, puis automatisable)

In [70]:
def extract_auto_metadata(text: str, filename: str) -> dict:
    """
    Extrait automatiquement la source et la langue.
    """
    header = text[:1000]
    
    # Déterminer la source
    source = "Archive historique"
    if "wikisource" in header.lower():
        source = "Wikisource"
    elif "gallica" in header.lower():
        source = "Gallica BnF"
    elif "journal officiel" in header.lower():
        source = "Journal Officiel"
    elif "convention" in filename.lower() or "déclaration" in filename.lower():
        source = "Document officiel"
    
    # Déterminer la langue
    language = "fr"
    english_words = re.findall(r'\b(the|and|of|in|to|for|with|that|this|from)\b', header[:500], re.IGNORECASE)
    french_words = re.findall(r'\b(le|la|de|et|les|des|un|une|dans|pour|par|sur)\b', header[:500], re.IGNORECASE)
    
    if len(english_words) > len(french_words) * 2:
        language = "en"
    
    return {
        "source": source,
        "language": language
    }


# Scan automatique du dossier raw/
docs = []
doc_counter = 1

for txt_file in sorted(RAW.glob("*.txt")):
    # Extraire le titre du nom de fichier (remplacer _ par espace)
    title = txt_file.stem.replace("_", " ")
    
    # Générer un doc_id unique
    doc_id = f"doc{doc_counter:03d}"
    
    # Lire le contenu pour extraire source et langue
    try:
        text_content = txt_file.read_text(encoding="utf-8", errors="ignore")
        metadata = extract_auto_metadata(text_content, txt_file.stem)
    except Exception as e:
        print(f"Erreur lecture {txt_file.name}: {e}")
        metadata = {"source": "Archive historique", "language": "fr"}
    
    # Créer l'entrée du document
    doc_entry = {
        "doc_id": doc_id,
        "title": title,
        "date": "",  # À remplir manuellement
        "author": "",  # À remplir manuellement
        "source": metadata["source"],
        "url": "",  # À remplir manuellement
        "local_path": str(txt_file.as_posix()),
        "language": metadata["language"],
    }
    
    docs.append(doc_entry)
    doc_counter += 1
    print(f"{doc_id}: {title} | Source: {metadata['source']} | Langue: {metadata['language']}")

# Créer le DataFrame et sauvegarder
df_docs = pd.DataFrame(docs, columns=DOC_SCHEMA)
df_docs.to_csv(DOCS_CSV, index=False, encoding='utf-8')

print(f"\nTotal: {len(docs)} documents")
print("A completer manuellement: date, author, url")
df_docs

doc001: Armistice à Bordeaux | Source: Wikisource | Langue: fr
doc002: Convention d’armistice franco-allemande | Source: Wikisource | Langue: fr
doc003: Déclaration interalliée du 17 décembre 1942 | Source: Wikisource | Langue: fr
doc004: Le racisme hitlérien, machine de guerre contre la France | Source: Wikisource | Langue: fr
doc005: Les services statistiques français pendant l’Occupation | Source: Wikisource | Langue: fr
doc006: Témoignage (Lebrun) | Source: Wikisource | Langue: fr

Total: 6 documents
A completer manuellement: date, author, url


,doc_id,title,date,author,source,url,local_path,language
0,doc001,Armistice à Bordeaux,,,Wikisource,,data/raw/Armistice_à_Bordeaux.txt,fr
1,doc002,Convention d’armistice franco-allemande,,,Wikisource,,data/raw/Convention_d’armistice_franco-alleman...,fr
2,doc003,Déclaration interalliée du 17 décembre 1942,,,Wikisource,,data/raw/Déclaration_interalliée_du_17_décembr...,fr
3,doc004,"Le racisme hitlérien, machine de guerre contre...",,,Wikisource,,"data/raw/Le_racisme_hitlérien,_machine_de_guer...",fr
4,doc005,Les services statistiques français pendant l’O...,,,Wikisource,,data/raw/Les_services_statistiques_français_pe...,fr
5,doc006,Témoignage (Lebrun),,,Wikisource,,data/raw/Témoignage_(Lebrun).txt,fr


# Cellule 3 — Fonctions utilitaires (lecture + hashing)

In [71]:
import unicodedata

def safe_filename(filename: str) -> str:
    """
    Normalise un nom de fichier pour éviter les problèmes de caractères spéciaux.
    Convertit les caractères accentués/spéciaux en ASCII safe.
    """
    # Décompose les caractères Unicode (é -> e + ´)
    nfd = unicodedata.normalize('NFD', filename)
    # Garde uniquement les caractères ASCII
    ascii_str = nfd.encode('ascii', 'ignore').decode('ascii')
    
    # Remplace les caractères problématiques
    replacements = {
        ' ': '_',
        '−': '-',
        '–': '-',
        '—': '-',
        ''': "'",
        ''': "'",
        '"': '"',
        '"': '"',
        '(': '_',
        ')': '_',
        ',': '_',
    }
    
    for old, new in replacements.items():
        ascii_str = ascii_str.replace(old, new)
    
    # Supprime les caractères multiples
    ascii_str = re.sub(r'_+', '_', ascii_str)
    ascii_str = re.sub(r'-+', '-', ascii_str)
    
    return ascii_str.strip('_-')

def file_sha1(path: Path) -> str:
    h = hashlib.sha1()
    with path.open("rb") as f:
        for chunk in iter(lambda: f.read(8192), b""):
            h.update(chunk)
    return h.hexdigest()

def read_text(path: Path) -> str:
    return path.read_text(encoding="utf-8", errors="ignore")

# Cellule 4 — Nettoyage

In [72]:
def clean_text(text: str) -> str:
    # Supprimer les lignes HTML/meta
    lines = text.split('\n')
    filtered_lines = []
    for line in lines:
        stripped = line.strip()
        # Ignorer les balises HTML/meta
        if stripped.startswith('<meta') or stripped.startswith('<link') or stripped.startswith('<!DOCTYPE'):
            continue
        filtered_lines.append(line)
    
    text = '\n'.join(filtered_lines)
    
    # Couper tout après "À propos de cette édition électronique" et variantes
    cutoff_phrases = [
        "À propos de cette édition électronique",
        "à propos de cette édition électronique",
        "À PROPOS DE CETTE ÉDITION",
        "Voir aussi",
        "Notes et références",
        "Source",
        "Cette édition électronique",
    ]
    
    for phrase in cutoff_phrases:
        idx = text.find(phrase)
        if idx != -1:
            text = text[:idx]
            break  # Couper à la première occurrence trouvée
    
    # Normalisation basique
    text = text.replace("\r\n", "\n").replace("\r", "\n")
    text = re.sub(r"[ \t]+", " ", text)           # espaces multiples
    text = re.sub(r"\n{3,}", "\n\n", text)        # trop de sauts de ligne
    text = text.strip()

    # Retirer césures de fin de ligne "exem-\nple" -> "exemple"
    text = re.sub(r"(\w)-\n(\w)", r"\1\2", text)

    return text

# Cellule 5 — Appliquer nettoyage à tous les docs

In [73]:
df_docs = pd.read_csv(DOCS_CSV)

for _, row in df_docs.iterrows():
    raw_path = Path(row["local_path"])
    if not raw_path.exists():
        print("MISSING:", raw_path)
        continue

    txt = read_text(raw_path)
    txt_clean = clean_text(txt)

    # Normaliser le nom de fichier pour éviter les SKIP
    safe_name = safe_filename(raw_path.stem) + ".txt"
    clean_path = CLEAN / safe_name
    clean_path.write_text(txt_clean, encoding="utf-8")
    print(f"{raw_path.name} -> {safe_name}")

print(f"\nCleaned files in: {CLEAN}")

Armistice_à_Bordeaux.txt -> Armistice_a_Bordeaux.txt
Convention_d’armistice_franco-allemande.txt -> Convention_darmistice_franco-allemande.txt
Déclaration_interalliée_du_17_décembre_1942.txt -> Declaration_interalliee_du_17_decembre_1942.txt
Le_racisme_hitlérien,_machine_de_guerre_contre_la_France.txt -> Le_racisme_hitlerien_machine_de_guerre_contre_la_France.txt
Les_services_statistiques_français_pendant_l’Occupation.txt -> Les_services_statistiques_francais_pendant_lOccupation.txt
Témoignage_(Lebrun).txt -> Temoignage_Lebrun.txt

Cleaned files in: data\clean


# Cellule 6 — Chunking (char-based "safe" + chevauchement)

Sans dépendances externes. Plus tard vous pourrez passer en token-based si vous voulez.

In [74]:
def chunk_text(text: str, chunk_size: int = 2000, overlap: int = 300):
    """
    Découpe par paragraphes/phrases avec overlap.
    chunk_size : 2000 chars ~ 300-500 tokens
    overlap : 300 chars ~ 50-75 tokens pour continuité contextuelle
    """
    chunks = []
    n = len(text)
    start = 0
    
    while start < n:
        end = min(n, start + chunk_size)
        
        # Stratégie de découpe (par ordre de priorité)
        window = text[start:end]
        
        # Priorité max: paragraphe (double saut de ligne)
        para_cut = window.rfind("\n\n")
        if para_cut > int(0.5 * len(window)):
            end = start + para_cut + 2  # Garder les \n\n
        
        # Sinon: fin de phrase (. ! ?)
        elif any(marker in window for marker in [". ", "! ", "? "]):
            sentence_cuts = [
                window.rfind(". "),
                window.rfind("! "),
                window.rfind("? ")
            ]
            best_cut = max(sentence_cuts)
            if best_cut > int(0.4 * len(window)):
                end = start + best_cut + 2  # Inclure ponctuation + espace
        
        # Fallback: simple saut de ligne
        elif "\n" in window:
            line_cut = window.rfind("\n")
            if line_cut > int(0.3 * len(window)):
                end = start + line_cut + 1
        
        # Extraire et nettoyer le chunk
        chunk = text[start:end].strip()
        
        if chunk:
            chunks.append((start, end, chunk))
        
        # Avancer avec overlap (sauf si on est à la fin)
        if end >= n:
            break
        start = max(end - overlap, start + 1)
    
    return chunks

# Cellule 7 — Génération chunks.jsonl avec métadonnées + citations

In [75]:
def make_chunk_id(doc_id: str, idx: int) -> str:
    return f"{doc_id}_{idx:04d}"

df_docs = pd.read_csv(DOCS_CSV)
out = CHUNKS_JSONL.open("w", encoding="utf-8")

total_chunks = 0
skipped = 0

for _, row in df_docs.iterrows():
    raw_path = Path(row["local_path"])
    
    # Utiliser le nom normalisé pour chercher le fichier clean
    safe_name = safe_filename(raw_path.stem) + ".txt"
    clean_path = CLEAN / safe_name
    
    if not clean_path.exists():
        print(f"SKIP (no clean): {clean_path}")
        skipped += 1
        continue

    text = read_text(clean_path)
    spans = chunk_text(text, chunk_size=2000, overlap=200)

    for i, (s, e, chunk) in enumerate(spans):
        rec = {
            "chunk_id": make_chunk_id(row["doc_id"], i),
            "doc_id": row["doc_id"],
            "text": chunk,
            "title": row.get("title", ""),
            "date": row.get("date", ""),
            "author": row.get("author", ""),
            "source": row.get("source", ""),
            "url": row.get("url", ""),
            "start_char": int(s),
            "end_char": int(e),
        }
        out.write(json.dumps(rec, ensure_ascii=False) + "\n")
        total_chunks += 1

out.close()
print(f"\nTotal chunks: {total_chunks}")
if skipped > 0:
    print(f"Skipped: {skipped} documents")
else:
    print("0 SKIP")
print(f"Wrote: {CHUNKS_JSONL}")


Total chunks: 1472
0 SKIP
Wrote: data\chunks.jsonl


# Cellule 8 — Sanity checks (stats + exemples)

In [76]:
# Charger les chunks
rows = []
with CHUNKS_JSONL.open("r", encoding="utf-8") as f:
    for line in f:
        rows.append(json.loads(line))

df_chunks = pd.DataFrame(rows)
df_chunks["len_chars"] = df_chunks["text"].str.len()

# STATISTIQUES GLOBALES
print("=" * 60)
print("CONTROLE QUALITE - DATASET RAG")
print("=" * 60)

# Nombre de documents et chunks
nb_docs = df_chunks["doc_id"].nunique()
nb_chunks = len(df_chunks)
print(f"\nDocuments sources    : {nb_docs}")
print(f"Chunks generes       : {nb_chunks}")
print(f"Moyenne chunks/doc   : {nb_chunks / nb_docs:.1f}")

# STATISTIQUES DE LONGUEUR
print("\n" + "-" * 60)
print("DISTRIBUTION DES LONGUEURS (caracteres)")
print("-" * 60)
print(f"Moyenne              : {df_chunks['len_chars'].mean():.0f} chars")
print(f"Mediane              : {df_chunks['len_chars'].median():.0f} chars")
print(f"Min                  : {df_chunks['len_chars'].min():.0f} chars")
print(f"Max                  : {df_chunks['len_chars'].max():.0f} chars")
print(f"Ecart-type           : {df_chunks['len_chars'].std():.0f} chars")

# Estimation en tokens (approximation : 1 token ~ 4 chars)
print(f"\nEstimation tokens    : {df_chunks['len_chars'].mean() / 4:.0f} tokens (moyenne)")

# REPARTITION PAR DOCUMENT
print("\n" + "-" * 60)
print("REPARTITION PAR DOCUMENT")
print("-" * 60)
chunks_per_doc = df_chunks.groupby("doc_id").size().sort_values(ascending=False)
for doc_id, count in chunks_per_doc.items():
    title = df_chunks[df_chunks["doc_id"] == doc_id]["title"].iloc[0]
    # Gérer les titres vides ou NaN
    title_str = str(title) if pd.notna(title) and title else doc_id
    print(f"{doc_id} : {count:3d} chunks - {title_str[:50]}")

# EXEMPLES ALEATOIRES
print("\n" + "=" * 60)
print("EXEMPLES ALEATOIRES (verification manuelle)")
print("=" * 60)

sample_chunks = df_chunks.sample(min(3, len(df_chunks)))
for idx, (_, chunk) in enumerate(sample_chunks.iterrows(), 1):
    print(f"\n--- CHUNK {idx} ---")
    print(f"ID       : {chunk['chunk_id']}")
    print(f"Document : {chunk['title'] if pd.notna(chunk['title']) and chunk['title'] else chunk['doc_id']}")
    print(f"Date     : {chunk['date'] if pd.notna(chunk['date']) and chunk['date'] else 'N/A'}")
    print(f"Source   : {chunk['source']}")
    print(f"Longueur : {chunk['len_chars']} chars")
    print(f"Position : [{chunk['start_char']} - {chunk['end_char']}]")
    print(f"\nExtrait (200 premiers caracteres):")
    print(f"{chunk['text'][:200]}...")

print("\n" + "=" * 60)

CONTROLE QUALITE - DATASET RAG

Documents sources    : 6
Chunks generes       : 1472
Moyenne chunks/doc   : 245.3

------------------------------------------------------------
DISTRIBUTION DES LONGUEURS (caracteres)
------------------------------------------------------------
Moyenne              : 677 chars
Mediane              : 161 chars
Min                  : 1 chars
Max                  : 1999 chars
Ecart-type           : 787 chars

Estimation tokens    : 169 tokens (moyenne)

------------------------------------------------------------
REPARTITION PAR DOCUMENT
------------------------------------------------------------
doc006 : 542 chunks - Témoignage (Lebrun)
doc005 : 284 chunks - Les services statistiques français pendant l’Occup
doc004 : 212 chunks - Le racisme hitlérien, machine de guerre contre la 
doc001 : 209 chunks - Armistice à Bordeaux
doc002 : 156 chunks - Convention d’armistice franco-allemande
doc003 :  69 chunks - Déclaration interalliée du 17 décembre 1942

EXEMPL

# Cellule 9 — Checklist

In [77]:
assert DOCS_CSV.exists()
assert CHUNKS_JSONL.exists()

# Vérifier qu'on a bien des URLs/sources pour citer
missing_sources = (df_chunks["source"].fillna("") == "") | (df_chunks["url"].fillna("") == "")
print("Chunks sans source ou url:", missing_sources.sum(), "/", len(df_chunks))

# Vérifier que les chunks ne sont pas vides
assert (df_chunks["text"].str.strip().str.len() > 0).all()

print("Dataset prêt pour indexation.")

Chunks sans source ou url: 1472 / 1472
Dataset prêt pour indexation.
